In [83]:
import psycopg2

In [84]:
try:
    conn = psycopg2.connect("dbname='postgres' user='devuser' host='localhost' password='devuser'")
except:
    print("I am unable to connect to the database")

In [85]:
cur = conn.cursor()

In [162]:
cur.execute("ROLLBACK")
conn.commit()

In [151]:
cur.execute("""DROP TABLE modules, folders""")
conn.commit()

In [154]:
cur.execute("""
CREATE TABLE folders (
    id SERIAL PRIMARY KEY,
    name VARCHAR(255),
    parent INT NULL REFERENCES folders(id))
""")

conn.commit()

In [155]:
cur.execute("""
CREATE TABLE modules (
    id SERIAL PRIMARY KEY,
    name VARCHAR(255),
    parent INT REFERENCES folders(id))
""")

conn.commit()

In [163]:
cur.execute("""
INSERT INTO folders (id, name)
VALUES (DEFAULT, '{folder_name}')
RETURNING id
""".format(folder_name="Folder1"))

folder_1_id = cur.fetchone()[0]

conn.commit()

cur.execute("""
INSERT INTO folders (id, name)
VALUES (DEFAULT, '{folder_name}')
RETURNING id
""".format(folder_name="Folder2"))

folder_2_id = cur.fetchone()[0]

conn.commit()

cur.execute("""
INSERT INTO folders (id, name, parent)
VALUES (DEFAULT, '{folder_name}', {parent_folder_id})
RETURNING id
""".format(folder_name="Folder11", parent_folder_id=folder_1_id))

folder_1_1_id = cur.fetchone()[0]

conn.commit()

cur.execute("""
INSERT INTO folders (id, name, parent)
VALUES (DEFAULT, '{folder_name}', {parent_folder_id})
""".format(folder_name="Folder111", parent_folder_id=folder_1_1_id))

conn.commit()

cur.execute("""
INSERT INTO folders (id, name, parent)
VALUES (DEFAULT, '{folder_name}', {parent_folder_id})
RETURNING id
""".format(folder_name="Folder21", parent_folder_id=folder_2_id))

conn.commit()

cur.execute("""
INSERT INTO folders (id, name, parent)
VALUES (DEFAULT, '{folder_name}', {parent_folder_id})
RETURNING id
""".format(folder_name="Folder22", parent_folder_id=folder_2_id))

conn.commit()

In [133]:
def print_with_indent(text, indent=0):
    for current_indent in range(0, indent):
        print("    ", end=" ")
    
    print(text)

In [164]:
def list(parent=None, indent=0):
    try:
        if(parent is None):
            cur.execute("""
            SELECT * FROM folders WHERE parent IS NULL
            """)
        else:
            cur.execute("""
            SELECT * FROM folders WHERE parent={query_parent}
            """.format(query_parent=parent))
    except:
        print("I cannot execute")

    try:
        rows = cur.fetchall()

        for row in rows:
            id = row[0]
            name = row[1]
            parent = row[2]

            print_with_indent(name + "\n", indent)

            list(id, indent + 1)
    except:
        pass

list()

Folder1

     Folder11

          Folder111

Folder2

     Folder21

     Folder22

